# Annotate the Normalised key point values in real time

In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hand Landmark Detection model
mp_hands = mp.solutions.hands.Hands()

# Open the camera
cap = cv2.VideoCapture(0)  # 0 for default camera

while True:
    # Read a frame from the camera
    success, frame = cap.read()

    # Process the frame with MediaPipe Hand Landmark Detection
    results = mp_hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # If hand landmarks are detected
    if results.multi_hand_landmarks:
        # Get the first detected hand
        hand_landmarks = results.multi_hand_landmarks[0]

        # Draw landmarks and annotate with normalized values
        for id, landmark in enumerate(hand_landmarks.landmark):
            x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(frame, f"{landmark.x:.2f}, {landmark.y:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the annotated frame
    cv2.imshow("Hand Landmarks", frame)

    # Check for the 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Hand Landmark Detection model
mp_hands = mp.solutions.hands.Hands()

# Open the camera
cap = cv2.VideoCapture(0)  # 0 for default camera

# Create directory to save captured photos
os.makedirs("photo_captured", exist_ok=True)

while True:
    # Read a frame from the camera
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the image horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Process the frame with MediaPipe Hand Landmark Detection
    results = mp_hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Convert the image color back so it can be displayed with OpenCV
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        # Get the first detected hand
        hand_landmarks = results.multi_hand_landmarks[0]

        # Get the bounding box coordinates
        x_coords = [landmark.x for landmark in hand_landmarks.landmark]
        y_coords = [landmark.y for landmark in hand_landmarks.landmark]
        x_min = int(min(x_coords) * frame.shape[1])
        x_max = int(max(x_coords) * frame.shape[1])
        y_min = int(min(y_coords) * frame.shape[0])
        y_max = int(max(y_coords) * frame.shape[0])

        # Draw the bounding box with different color
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)

        # Draw scale on the bounding box
        for i in range(11):
            x_pos = x_min + i * (x_max - x_min) // 10
            y_pos = y_min + i * (y_max - y_min) // 10
            cv2.line(image, (x_pos, y_min), (x_pos, y_min - 10), (0, 255, 255), 1)
            cv2.line(image, (x_min, y_pos), (x_min - 10, y_pos), (0, 255, 255), 1)
            # Mark the values on the axis lines
            cv2.putText(image, f'{i/10:.1f}', (x_pos, y_min - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)
            cv2.putText(image, f'{i/10:.1f}', (x_min - 25, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)

        # Draw landmarks and annotate with normalized values
        for id, landmark in enumerate(hand_landmarks.landmark):
            x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(image, f"{landmark.x:.2f}, {landmark.y:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the annotated frame
    cv2.imshow("Hand Landmarks", image)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        # Save the current frame with annotations
        filename = f"photo_captured/frame_{cv2.getTickCount()}.png"
        cv2.imwrite(filename, image)
        print(f"Captured and saved {filename}")

# Release resources
cap.release()
cv2.destroyAllWindows()


# 2nd version

# Error Corrected

# press "S" to capture and save the frame at any time

In [3]:
import cv2
import mediapipe as mp

# Initialize MediaPipe hands and drawing modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Set up MediaPipe Hands with default parameters
with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image and detect hands
        results = hands.process(image)

        # Convert the image color back so it can be displayed with OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get the bounding box coordinates
                x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                x_min = int(min(x_coords) * frame.shape[1])
                x_max = int(max(x_coords) * frame.shape[1])
                y_min = int(min(y_coords) * frame.shape[0])
                y_max = int(max(y_coords) * frame.shape[0])

                # Draw the bounding box
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                # Draw scale on the bounding box
                for i in range(11):
                    x_pos = x_min + i * (x_max - x_min) // 10
                    y_pos = y_min + i * (y_max - y_min) // 10
                    cv2.line(image, (x_pos, y_min), (x_pos, y_min - 10), (0, 255, 0), 1)
                    cv2.line(image, (x_min, y_pos), (x_min - 10, y_pos), (0, 255, 0), 1)

                # Optionally, draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Display normalized keypoints
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    x = landmark.x
                    y = landmark.y
                    x_disp = int(x * frame.shape[1])
                    y_disp = int(y * frame.shape[0])
                    cv2.putText(image, f'{idx}: ({x:.2f}, {y:.2f})', (x_disp, y_disp), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1, cv2.LINE_AA)

        # Display the resulting frame
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import mediapipe as mp
import os

# Initialize MediaPipe hands and drawing modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create directory to save captured photos
os.makedirs("photo_captured", exist_ok=True)

# Set up MediaPipe Hands with default parameters
with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image and detect hands
        results = hands.process(image)

        # Convert the image color back so it can be displayed with OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get the bounding box coordinates
                x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                x_min = int(min(x_coords) * frame.shape[1])
                x_max = int(max(x_coords) * frame.shape[1])
                y_min = int(min(y_coords) * frame.shape[0])
                y_max = int(max(y_coords) * frame.shape[0])

                # Draw the bounding box with different color
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)

                # Draw scale on the bounding box
                for i in range(11):
                    x_pos = x_min + i * (x_max - x_min) // 10
                    y_pos = y_min + i * (y_max - y_min) // 10
                    cv2.line(image, (x_pos, y_min), (x_pos, y_min - 10), (0, 255, 255), 1)
                    cv2.line(image, (x_min, y_pos), (x_min - 10, y_pos), (0, 255, 255), 1)
                    # Mark the values on the axis lines
                    cv2.putText(image, f'{i/10:.1f}', (x_pos, y_min - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)
                    cv2.putText(image, f'{i/10:.1f}', (x_min - 25, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)

                # Optionally, draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Display normalized keypoints with different color
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    x = landmark.x
                    y = landmark.y
                    x_disp = int(x * frame.shape[1])
                    y_disp = int(y * frame.shape[0])
                    cv2.putText(image, f'{idx}: ({x:.2f}, {y:.2f})', (x_disp, y_disp), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)

        # Display the resulting frame
        cv2.imshow('Hand Tracking', image)

        key = cv2.waitKey(5) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('c'):
            # Save the current frame
            filename = f"photo_captured/frame_{cv2.getTickCount()}.png"
            cv2.imwrite(filename, frame)
            print(f"Captured and saved {filename}")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import mediapipe as mp
import os

# Initialize MediaPipe hands and drawing modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create directory to save captured photos
os.makedirs("photo_captured", exist_ok=True)

# Set up MediaPipe Hands with default parameters
with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image and detect hands
        results = hands.process(image)

        # Convert the image color back so it can be displayed with OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get the bounding box coordinates
                x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                x_min = min(x_coords)
                x_max = max(x_coords)
                y_min = min(y_coords)
                y_max = max(y_coords)

                # Draw the bounding box with different color
                cv2.rectangle(image, (int(x_min * frame.shape[1]), int(y_min * frame.shape[0])), 
                     (int(x_max * frame.shape[1]), int(y_max * frame.shape[0])), (0, 255, 255), 2)

                # Draw scale on the bounding box
                for i in range(11):
                    x_pos = x_min + i * (x_max - x_min) / 10
                    y_pos = y_min + i * (y_max - y_min) / 10
                    cv2.line(image, (int(x_pos * frame.shape[1]), int(y_min * frame.shape[0])), 
                                    (int(x_pos * frame.shape[1]), int((y_min - 0.1) * frame.shape[0])), (0, 255, 255), 1)
                    cv2.line(image, (int(x_min * frame.shape[1]), int(y_pos * frame.shape[0])), 
                                    (int((x_min - 0.1) * frame.shape[1]), int(y_pos * frame.shape[0])), (0, 255, 255), 1)
                    # Mark the values on the axis lines
                    cv2.putText(image, f'{i/10:.1f}', (int(x_pos * frame.shape[1]), int((y_min - 0.15) * frame.shape[0])), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)
                    cv2.putText(image, f'{i/10:.1f}', (int((x_min - 0.25) * frame.shape[1]), int(y_pos * frame.shape[0])), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)

                # Optionally, draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Display normalized keypoints with different color
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    x_disp = int(landmark.x * frame.shape[1])
                    y_disp = int(landmark.y * frame.shape[0])
                    cv2.putText(image, f'{idx}: ({landmark.x:.2f}, {landmark.y:.2f})', (x_disp, y_disp), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                    
        # Display the resulting frame
        cv2.imshow('Hand Tracking', image)

        key = cv2.waitKey(5) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('c'):
            # Save the current frame with annotations
            filename = f"photo_captured/frame_{cv2.getTickCount()}.png"
            cv2.imwrite(filename, image)  # Save the image with annotations
            print(f"Captured and saved {filename}")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe hands and drawing modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create directory to save captured photos
os.makedirs("photo_captured", exist_ok=True)

# Set up MediaPipe Hands with default parameters
with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image and detect hands
        results = hands.process(image)

        # Convert the image color back so it can be displayed with OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get the bounding box coordinates
                x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                x_min = int(min(x_coords) * frame.shape[1])
                x_max = int(max(x_coords) * frame.shape[1])
                y_min = int(min(y_coords) * frame.shape[0])
                y_max = int(max(y_coords) * frame.shape[0])

                # Draw the bounding box with different color
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)

                # Draw scale on the bounding box
                for i in range(11):
                    x_pos = x_min + i * (x_max - x_min) // 10
                    y_pos = y_min + i * (y_max - y_min) // 10
                    cv2.line(image, (x_pos, y_min), (x_pos, y_min - 10), (0, 255, 255), 1)
                    cv2.line(image, (x_min, y_pos), (x_min - 10, y_pos), (0, 255, 255), 1)
                    # Mark the values on the axis lines
                    cv2.putText(image, f'{i/10:.1f}', (x_pos, y_min - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)
                    cv2.putText(image, f'{i/10:.1f}', (x_min - 25, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)

                # Optionally, draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Display normalized keypoints with different color
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    x = landmark.x
                    y = landmark.y
                    x_disp = int((x - min(x_coords)) / (max(x_coords) - min(x_coords)) * (x_max - x_min) + x_min)
                    y_disp = int((y - min(y_coords)) / (max(y_coords) - min(y_coords)) * (y_max - y_min) + y_min)
                    cv2.putText(image, f'{idx}: ({x:.2f}, {y:.2f})', (x_disp, y_disp), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)

        # Display the resulting frame
        cv2.imshow('Hand Tracking', image)

        key = cv2.waitKey(5) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('c'):
            # Save the current frame with annotations
            filename = f"photo_captured/frame_{cv2.getTickCount()}.png"
            cv2.imwrite(filename, image)  # Save the image with annotations
            print(f"Captured and saved {filename}")
        elif key == ord('s'):
            # Create a white background image
            white_image = 255 * np.ones_like(frame)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Get the bounding box coordinates
                    x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                    y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                    x_min = int(min(x_coords) * frame.shape[1])
                    x_max = int(max(x_coords) * frame.shape[1])
                    y_min = int(min(y_coords) * frame.shape[0])
                    y_max = int(max(y_coords) * frame.shape[0])

                    # Draw the bounding box with different color
                    cv2.rectangle(white_image, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)

                    # Draw scale on the bounding box
                    for i in range(11):
                        x_pos = x_min + i * (x_max - x_min) // 10
                        y_pos = y_min + i * (y_max - y_min) // 10
                        cv2.line(white_image, (x_pos, y_min), (x_pos, y_min - 10), (0, 255, 255), 1)
                        cv2.line(white_image, (x_min, y_pos), (x_min - 10, y_pos), (0, 255, 255), 1)
                        # Mark the values on the axis lines
                        cv2.putText(white_image, f'{i/10:.1f}', (x_pos, y_min - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)
                        cv2.putText(white_image, f'{i/10:.1f}', (x_min - 25, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)

                    # Draw hand landmarks on white background
                    mp_drawing.draw_landmarks(white_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Display normalized keypoints with different color
                    for idx, landmark in enumerate(hand_landmarks.landmark):
                        x = landmark.x
                        y = landmark.y
                        x_disp = int((x - min(x_coords)) / (max(x_coords) - min(x_coords)) * (x_max - x_min) + x_min)
                        y_disp = int((y - min(y_coords)) / (max(y_coords) - min(y_coords)) * (y_max - y_min) + y_min)
                        cv2.putText(white_image, f'{idx}: ({x:.2f}, {y:.2f})', (x_disp, y_disp), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)

            # Save the skeleton image
            skeleton_filename = f"photo_captured/skeleton_{cv2.getTickCount()}.png"
            cv2.imwrite(skeleton_filename, white_image)  # Save the skeleton image
            print(f"Captured and saved {skeleton_filename}")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Captured and saved photo_captured/skeleton_46884224307700.png
